In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectPercentile, VarianceThreshold, f_classif
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.preprocessing import FunctionTransformer
from copy import copy

import os
import lightgbm as lgb
from sklearn import preprocessing

In [2]:
def get_model():
    exported_pipeline = make_pipeline(
        make_union(
            make_pipeline(
                make_union(
                    FunctionTransformer(copy),
                    FunctionTransformer(copy)
                ),
                SelectPercentile(score_func=f_classif, percentile=18)
            ),
            FunctionTransformer(copy)
        ),
        StackingEstimator(estimator=SGDClassifier(alpha=0.01, eta0=0.1, fit_intercept=False, l1_ratio=1.0, learning_rate="constant", loss="hinge", penalty="elasticnet", power_t=0.1)),
        VarianceThreshold(threshold=0.05),
        ExtraTreesClassifier(bootstrap=False, criterion="entropy", max_features=0.55, min_samples_leaf=1, min_samples_split=4, n_estimators=100)
    )
    set_param_recursive(exported_pipeline.steps, 'random_state', 42)
    return exported_pipeline

In [3]:
def rmsle_cv(model=None, X_train=None, y_train=None):
    seed = 42
    n_folds = 5
    kf = KFold(n_folds, shuffle=True, random_state=seed).get_n_splits(X_train)
    score = cross_val_score(model, X_train, y_train,
                           scoring="f1_macro", cv = kf)
    return score

In [4]:
def feature_generate_manually():
    train_path = '../data/hy_round1_train_20200102'
    test_path = '../data/hy_round1_testA_20200102'

    train_df_list = []
    for file_name in os.listdir(train_path):
        df = pd.read_csv(os.path.join(train_path, file_name))
        train_df_list.append(df)

    test_df_list = []
    for file_name in os.listdir(test_path):
        df = pd.read_csv(os.path.join(test_path, file_name))
        test_df_list.append(df)

    train_df = pd.concat(train_df_list)
    test_df = pd.concat(test_df_list)

    train_df['time'] = pd.to_datetime(train_df['time'], format='%m%d %H:%M:%S')
    test_df['time'] = pd.to_datetime(test_df['time'], format='%m%d %H:%M:%S')

    all_df = pd.concat([train_df, test_df], sort=False)
    
    new_df = all_df.groupby('渔船ID').agg(x_min=('x', 'min'), x_max=('x', 'max'), x_mean=('x', 'mean'), x_std=('x', 'std'), x_skew=('x', 'skew'), x_sum=('x', 'sum'),
                y_min=('y', 'min'), y_max=('y', 'max'), y_mean=('y', 'mean'), y_std=('y', 'std'), y_skew=('y', 'skew'), y_sum=('y', 'sum'),
                v_min=('速度', 'min'), v_max=('速度', 'max'), v_mean=('速度', 'mean'), v_std=('速度', 'std'), v_skew=('速度', 'skew'), v_sum=('速度', 'sum'),
                d_min=('方向', 'min'), d_max=('方向', 'max'), d_mean=('方向', 'mean'), d_std=('方向', 'std'), d_skew=('方向', 'skew'), d_sum=('方向', 'sum'))
    new_df['x_max-x_min'] = new_df['x_max'] - new_df['x_min']
    new_df['y_max-y_min'] = new_df['y_max'] - new_df['y_min']
    new_df['x_max-y_min'] = new_df['x_max'] - new_df['y_min']
    new_df['y_max-x_min'] = new_df['y_max'] - new_df['x_min']
    
    new_df['slope'] = new_df['y_max-y_min'] / np.where(new_df['x_max-x_min']==0, 0.001, new_df['x_max-x_min'])
    new_df['area'] = new_df['x_max-x_min'] * new_df['y_max-y_min']
    
    new_df['type'] = all_df.groupby('渔船ID').agg(type=('type', 'first'))['type'].values
    
    print(new_df.drop(columns=['type']).columns)
    
    X_train = new_df.drop(columns=['type']).iloc[:7000].values
    y_train = new_df.iloc[:7000]['type'].values

    X_test = new_df.drop(columns=['type']).iloc[7000:].values

    return X_train, y_train, X_test, new_df.drop(columns=['type']).columns

In [5]:
def feature_generate_tsfresh():
    train_df = pd.read_csv('./train.csv')
    X_train = train_df.drop(columns=['type'])
    y_train = train_df['type']
    
    le = preprocessing.LabelEncoder()
    y_train = le.fit_transform(y_train)

    test_df = pd.read_csv('./test.csv')
    X_test = test_df[X_train.columns]
    
    base_model =  lgb.LGBMClassifier(n_estimators=400, objective='multiclass')
    base_model.fit(X_train.values, y_train)
    
    selected_columns = X_train.columns[np.argsort(base_model.feature_importances_)[::-1][:24]]
#     selected_columns = selected_columns[~np.isin(selected_columns, ['x__minimum', 'y__maximum', 'y__minimum', 'x__maximum'])]
    print(selected_columns)
    
    X_train = X_train[selected_columns].values

    X_test = X_test[selected_columns]
    for column in list(X_test.columns[X_test.isnull().sum() > 0]):
        mean_val = X_test[column].mean()
        X_test[column].fillna(mean_val, inplace=True)
    X_test = X_test.values
    
    return X_train, le.inverse_transform(y_train), X_test, selected_columns

In [6]:
X_train_manually, y_train, X_test_manually, feature_manually = feature_generate_manually()

Index(['x_min', 'x_max', 'x_mean', 'x_std', 'x_skew', 'x_sum', 'y_min',
       'y_max', 'y_mean', 'y_std', 'y_skew', 'y_sum', 'v_min', 'v_max',
       'v_mean', 'v_std', 'v_skew', 'v_sum', 'd_min', 'd_max', 'd_mean',
       'd_std', 'd_skew', 'd_sum', 'x_max-x_min', 'y_max-y_min', 'x_max-y_min',
       'y_max-x_min', 'slope', 'area'],
      dtype='object')


In [7]:
X_train_tsfresh, y_train, X_test_tsfresh, feature_tsfresh = feature_generate_tsfresh()

Index(['x__quantile__q_0.1', 'y__maximum', 'x__minimum', 'y__quantile__q_0.9',
       'x__quantile__q_0.2', 'y__quantile__q_0.8', 'y__quantile__q_0.7',
       'y__minimum', '速度__number_crossing_m__m_1', 'x__quantile__q_0.3',
       'x__maximum', 'x__quantile__q_0.9', '速度__quantile__q_0.7',
       '速度__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'y__number_cwt_peaks__n_1', 'x__quantile__q_0.4', 'y__quantile__q_0.6',
       '方向__ar_coefficient__k_10__coeff_0',
       '速度__fft_coefficient__coeff_6__attr_"real"', 'y__quantile__q_0.1',
       '速度__agg_autocorrelation__f_agg_"mean"__maxlag_40',
       '速度__ratio_beyond_r_sigma__r_2',
       '方向__fft_coefficient__coeff_64__attr_"abs"', 'y__quantile__q_0.4'],
      dtype='object')


In [8]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)

In [9]:
X_train = np.concatenate([X_train_tsfresh], axis=1)
X_test = np.concatenate([X_test_tsfresh], axis=1)

In [10]:
from sklearn.metrics import f1_score

def evaluate_macroF1_lgb(truth, predictions):  
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

In [11]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

kf = KFold(n_splits=5, random_state=42, shuffle=True)

model_list = []
score_list = []
for train_index, test_index in kf.split(X_train):
    model = get_model()
    eval_set = (X_train[test_index], y_train[test_index])
    model.fit(X_train[train_index], y_train[train_index])
    model_list.append(model)
    score_list.append(f1_score(y_train[test_index], model.predict(X_train[test_index]), average='macro'))
    
print(score_list)
print(np.mean(score_list), np.std(score_list))

[0.9102914602643932, 0.9297908609350584, 0.937409185635499, 0.9137327067272167, 0.9135158792115048]
0.9209480185547345 0.010677317430937786


In [12]:
X_test.shape

(2000, 24)

In [13]:
result_list = []
for model in model_list:
    result = model.predict_proba(X_test)
    result_list.append(result)

result = np.argmax(np.sum(np.array(result_list), axis=0) / 5, axis=1)

result = le.inverse_transform(result)
pd.DataFrame(result, index=range(7000, 9000)).to_csv('result.csv', header=None)